# Servir modelo con `TensorFlow Serving`

Para servir el modelo con `TensorFlow Serving` hay que iniciar un contenedor a partir de dicha imagen enlazando el modelo en local dentro del contenedor. Es importante que el modelo esté estructurado bajo una carpeta raíz que simboliza la versión de un modelo:

<center>
  <img src="https://i.ibb.co/4M9BR2P/Captura-de-pantalla-2024-05-12-040608.png" alt="Estructura modelo"/>
</center>

En este caso, se trata de la versión `1` del modelo. Por eso existe una carpeta con el mismo nombre. Si no fuera así y no existiera dicha carpeta, el modelo no se serviría correctamente dentro del contenedor de `Docker`. Una vez se tiene esto en cuenta, se inicia dicho contenedor a partir de la imagen de `TensorFlow Serving`:

Aquí se inicia un contenedor con nombre `sentiment_classifier` a partir de la imagen de `tensorflow/serving` enlazando el puerto `8501` del `localhost` del PC con el puerto `8501` del contenedor, ya que `TensorFlow Serving` expone por defecto el `endpoint` tipo `REST` en el puerto `8501`, por lo que se puede consultar dicho `endpoint` en el puerto `8501` de nuestro `localhost`. Como también se puede apreciar, se monta un volumen enlazando el directorio donde está el modelo almacenado en local con el directorio `/models/sentiment_classifier` dentro del contenedor.

Ahora se puede mandar una petición `POST` al puerto del `localhost` enlazado con el contenedor para obtener predicciones:

Si se quisiera hacer con código de `Python` sería algo parecido a esto:

In [79]:
# Librerías
import requests
import json
import tensorflow as tf

# URL local donde se sirven las predicciones del modelo
url = 'http://localhost:8501/v1/models/sentiment_classifier:predict'

# Header para contenido de tipo JSON
headers = {'Content-type': 'application/json'}

# Datos a los que realizar la inferencia
X_test = json.dumps({"instances": [['this is sad'], ['my dog is the best']]
                    })

# Solicitud 'POST' al puerto donde el modelo está siendo expuesto con el 'header' de contenido JSON
resp = requests.post(url, headers=headers, data=X_test)

# Imprime el contenido de la respuesta
print(resp.content)

b'{\n    "predictions": [[0.00952669699], [0.88457787]\n    ]\n}'


Como se puede apreciar, no se trata de un método amigable para realizar las predicciones, y por tanto, exportar a producción. Por lo tanto, se va a usar otra alternativa.

# Despliegue (producción)

## Docker

### Backend (Tensorflow Serving)

Para crear una imagen con el modelo servido en `TensorFlow Serving` hay que copiar dicho modelo fuera de un volumen, ya que si éste esta montado en un volumen, como en el caso anterior, al crear una imagen nueva, se pierde. Por tanto, se pueden realizar cualquiera de estas dos cosas:

- Hacer un `commit` a un contenedor donde el modelo esté copiado fuera de un volumen.
- Crear un `Dockerfile` en el que a partir de una imagen base de `tensorflow/serving` se copia el modelo dentro de una subcarpeta dentro de `models` y se declare como variable de entorno una con clave `MODEL_NAME` y con el valor de la subcarpeta de `models`, exponiendo el puerto donde se sirve el modelo en una `API REST`.

En este caso se opta por el segundo método, por lo que una vez creado el `Dockerfile` se crea la imagen con el siguiente comando:

Mediante el argumento `-t` se especifica el nombre y el `tag` de la imagen en formato `nombre:tag`; y mediante el argumento `-f` el archivo `Dockerfile` a usar. Por lo que la imagen creada se llama `tf_serving_sentiment_classifier` y el `tag` es `1.0` a partir del `Dockerfile` de la carpeta `app\backend`. El `.` del final indica el contexto del `build`, es decir, el directorio raíz a partir de donde `Docker` buscará los archivos (en este caso, la carpeta raíz del proyecto `Twitter_Sentiment_Analysis`).

Una vez se crea la imagen de `Docker`, ya se puede realizar peticiones `POST` al modelo dentro de un contenedor. Para ello se inicia uno de ellos a partir de la imagen creada, con un comando como el siguiente:

Mediante este comando se inicia un contenedor a partir de la imagen creada, enlazando el puerto `8501` de nuestro `localhost` con el puerto `8501` del contenedor (el puerto donde se encuentra la `API REST` dentro del contenedor para realizar peticiones `POST`). De esta forma, si se realiza peticiones a la dirección `127.0.0.1:8501` del `localhost` se obtendrán predicciones de los sentimientos de los `tweets`.

Después de construir la imagen de Docker, ésta se sube a `Dockerhub` para tenerlo almacenado en un repositorio y poder descargar la imagen posteriormente en cualquier entorno. Para ello, primero se crea un `tag` a la imagen de `Docker` creada para especificar cual es nuestro usuario `Dockerhub` y cual sería el nombre de la imagen y el `tag` en el repositorio. Por tanto, se ejecuta el siguiente comando:

Donde `xxxx` es el nombre de la cuenta de `Dockerhub`.

Una vez la imagen ya ha sido `taggeada`, se sube al repositorio de `Dockerhub`:

### Frontend (Streamlit)

En la parte de `frontend` se crea una `app` con `Streamlit` que haga consultas al `backend` de `TensorFlow Serving`. Una vez es creada se crea otro `Dockerfile` para crear una imagen que sirva la aplicación web.

Posteriormente se deben realizar los mismos pasos que en la imagen de `backend`, es decir: `tagear` la imagen y subirla a `Dockerhub` para que la imagen esté disponible a descargar.

## Kubernetes

`Kubernetes` es un sistema de orquestación de contenedores usado para el despliegue de contenedores de `Docker`. Está diseñado para administrar eficientemente y coordinar `clusters` y cargas de trabajo a gran escala en un entorno de producción. Además, ayuda a gestionar servicios contenerizados mediante la automatización en el despliegue.

De esta forma, se crea un despliegue con, por ejemplo, tres `pods`, cada uno de los cuales tendrá dos contenedores (uno proveniente de la imagen de la aplicación web; y otro proveniente de la imagen de la API). Aunque si es necesario, se puede escalar dicho despliegue a más o menos `pods` dependiendo del tráfico que requiera la API y la aplicación web. Además, estos `pods` se van a exponer a través de un servicio de tipo `LoadBalancer`, de forma que la API y la aplicación web se hacen accesibles a clientes externos que están fuera del `cluster` de `Kubernetes` y además añade funcionalidad de balanceo de carga para distribuir el tráfico entre los distintos `pods`, reduciendo los efectos negativos de sobrecarga.

Para ello se crea un archivo `YAML` que define tanto el servicio como el despliegue:

Como se puede apreciar, los puertos de los contenedores deben ser el `8000` para la API y el `8501` para la aplicación web, que es donde están alojadas la API y la aplicación web en cada una de las dos imágenes, y por tanto, los `targetPort` del servicio también debe ser los mismos, ya que estos serán los puertos a los que va a apuntar el servicio `LoadBalancer`.

En el cluster local de `Minikube`, una vez creado el despliegue y el servicio, para acceder a la API y a la aplicación web hay que ejecutar el siguiente comando que devuelve dos `URLs` (una para la API y otra para la aplicación web) para conectarse al servicio que expone dichas aplicaciones:

<p style="text-align: center;"> minikube -n airbnb service app-fullstack-airbnb-service </p>

Esto lo que hace es crear un túnel para el servicio `app-fullstack-airbnb-service`, creando un movimiento de datos de una red a otra. Es decir, se mueve el flujo de datos de la `IP` y de cada uno de los puertos del `nodo` de `Minikube` (`nodePort`) que usa el servicio, a la `IP` local y a dos puertos aleatorios del `localhost`, de forma que se puede consultar tanto la `API` como la aplicación web en el navegador local.